# Webscrapeo de escuelas - INDIA

In [1]:
import time
import requests
import numpy as np
import pandas as pd

## Funciones

In [2]:
def get_school_df(URL, newId, counter):
    """[Get School DataFrame by calling its API]

    Parameters
    ----------
    URL : [str]
        [String with the original URL]
    newId : [int]
        [Integer belonging to certain school]

    Returns
    -------
    [school_df]
        [DataFrame with the school's info]
    """
    
    new_API = URL.replace("objectIds=20", f"objectIds={newId}")
    r = requests.get(new_API)
    content = r.json()
    school_info = content['features'][0]['attributes']
    school_df = pd.DataFrame(school_info, index=[counter])
    school_df = school_df[['objectid_1', 'fid_school', 'objectid', 'schcd', 'schname', 'schcat',
       'school_cat', 'schtype', 'school_typ', 'schmgt', 'management', 'rururb', 'location', 
       'pincode', 'dtname', 'udise_stco', 'stname', 'longitude', 'latitude', 'stcode11', 'dtcode11']]
    
    school_df.rename(columns={"schcd": "school_code", "schname": "school_name"}, inplace=True)

    return school_df


## Parámetros

In [3]:
URL = """https://geoportal.nic.in/nicgis/rest/services/SCHOOLGIS/Schooldata/MapServer/0/query?f=json&returnGeometry=true&spatialRel=esriSpatialRelIntersects&objectIds=20&outFields=*&outSR=102100&quantizationParameters=%7B%22mode%22%3A%22view%22%2C%22originPosition%22%3A%22upperLeft%22%2C%22tolerance%22%3A4.777314267948289%2C%22extent%22%3A%7B%22xmin%22%3A68.5015470000764%2C%22ymin%22%3A6.8114540002900235%2C%22xmax%22%3A97.02722199976724%2C%22ymax%22%3A35.032117999820365%2C%22spatialReference%22%3A%7B%22wkid%22%3A4326%2C%22latestWkid%22%3A4326%7D%7D%7D"""

In [4]:
short_URL = "https://geoportal.nic.in/nicgis/rest/services/SCHOOLGIS/Schooldata/MapServer/0/query?f=json&returnGeometry=true&spatialRel=esriSpatialRelIntersects&objectIds=20"

## Pipeline

In [5]:
URL_2 = """https://geoportal.nic.in/nicgis/rest/services/SCHOOLGIS/Schooldata/MapServer/0/query?f=json&returnGeometry=true&spatialRel=esriSpatialRelIntersects&objectIds=0&outFields=*&outSR=102100&quantizationParameters=%7B%22mode%22%3A%22view%22%2C%22originPosition%22%3A%22upperLeft%22%2C%22tolerance%22%3A4.777314267948289%2C%22extent%22%3A%7B%22xmin%22%3A68.5015470000764%2C%22ymin%22%3A6.8114540002900235%2C%22xmax%22%3A97.02722199976724%2C%22ymax%22%3A35.032117999820365%2C%22spatialReference%22%3A%7B%22wkid%22%3A4326%2C%22latestWkid%22%3A4326%7D%7D%7D"""

In [6]:
r = requests.get(URL_2)
r.json()

{'displayFieldName': 'vilname',
 'transform': {'originPosition': 'upperLeft',
  'scale': [4.777314267948289, 4.777314267948289],
  'translate': [68.5015470000764, 35.032117999820365]},
 'fieldAliases': {'objectid_1': 'objectid_1',
  'fid_school': 'fid_school',
  'objectid': 'objectid',
  'schcd': 'schcd',
  'schname': 'School',
  'schcat': 'Cat_Code',
  'school_cat': 'Category',
  'schtype': 'Type Code',
  'school_typ': 'School Type',
  'schmgt': 'Mngt Code',
  'management': 'Management',
  'rururb': 'Rural_Urban',
  'location': 'Location',
  'pincode': 'Pincode',
  'dtname': 'District',
  'udise_stco': 'udise_stco',
  'stname': 'State',
  'vilname': 'Village',
  'longitude': 'longitude',
  'latitude': 'latitude',
  'stcode11': 'stcode11',
  'dtcode11': 'dtcode11'},
 'geometryType': 'esriGeometryPoint',
 'spatialReference': {'wkid': 102100, 'latestWkid': 3857},
 'fields': [{'name': 'objectid_1',
   'type': 'esriFieldTypeOID',
   'alias': 'objectid_1'},
  {'name': 'fid_school',
   'type

In [7]:
school_df = get_school_df(URL, 1489000, 1)
school_df

,objectid_1,fid_school,objectid,school_code,school_name,schcat,school_cat,schtype,school_typ,schmgt,...,rururb,location,pincode,dtname,udise_stco,stname,longitude,latitude,stcode11,dtcode11
1,1489000,632472,636979,37080101604,GMS AKCHAMAL,2,Upper Primary,2,Girls,1,...,1,,194103,Kargil,37,LADAKH,76.1629,34.5561,38,004


## Probar con 1000 escuelas

In [8]:
schools_df = pd.DataFrame([])
school_numbers = np.arange(1400001, 1405001)
for school_number in school_numbers:
    try:
        school_df = get_school_df(URL, school_number, school_number)
        schools_df = pd.concat([schools_df, school_df], ignore_index=True)
        time.sleep(np.random.randint(0, 2))
        
    except Exception as err:
        print(str(err))
        print("Unable to get data")
        school_df = pd.DataFrame([])
        schools_df = pd.concat([schools_df, school_df], ignore_index=True)
        time.sleep(2)

    if school_number % 1000 == 0:
        time.sleep(4)
        print(f"{school_number} schools obtained")
        schools_df.to_csv(f"schools_df_{school_number}.csv", index=False, encoding='utf-8-sig')



1401000 schools obtained
1402000 schools obtained
1403000 schools obtained
1404000 schools obtained
1405000 schools obtained


In [10]:
schools_df.to_csv(f"schools_df_{school_number}.csv", index=False, encoding='utf-8-sig')
